In [ ]:
! pip install langchain_openai langchain langchain_core -U

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import datetime

from google.colab import userdata

import os
os.environ["OPENAI_API_BASE"] = userdata.get('OPENAI_API_BASE')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# 初始化ChatOpenAI模型，使用qwen2.5-72b-instruct模型，温度设置为0
model = ChatOpenAI(model="qwen2.5-72b-instruct", temperature=0)

# 定义一个工具函数，用于获取用户的年龄
@tool
def get_user_age(name: str) -> str:
    """使用这个工具来查找用户的年龄。"""
    # 这是一个占位符，实际实现中会根据用户名返回年龄
    if "bob" in name.lower():
        return "42 years old"
    return "41 years old"
  
@tool
def get_current_time():
    """获取当前时间。"""
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

# 定义工具列表，包含get_user_age工具
tools = [get_user_age, get_current_time]

# 定义聊天提示模板
prompt = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{chat_history}"),  # 占位符，用于存储聊天历史
        ("human", "{input}"),  # 用户输入
        ("placeholder", "{agent_scratchpad}"),  # 占位符，用于存储代理的临时笔记
    ]
)

# 创建工具调用代理
agent = create_tool_calling_agent(model, tools, prompt)

# 实例化记忆对象，用于存储对话历史
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 创建代理执行器，将代理、工具和记忆对象结合起来
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,  # 将记忆对象传递给执行器
)

# 验证代理是否可以使用工具
res1 = agent_executor.invoke({"input": "hi! my name is bob what is my age?"})
print(res1)
print(res1['output'])
print()

# 验证代理是否可以访问对话历史
# 代理应该能够回答用户的名字是bob
res2 = agent_executor.invoke({"input": "do you remember my name?"})
print(res2)
print(res2['output'])

print()
res3 = agent_executor.invoke({"input": "当前时间是?"})
print(res3)
print(res3['output'])